In [1]:
import mlflow

In [2]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [3]:
mlflow.search_experiments()

[<Experiment: artifact_location='/workspaces/mlops-camp/notebook/03-experiment-tracking/running-mlflow-exp/artifacts_local/1', creation_time=1716837118065, experiment_id='1', last_update_time=1716837118065, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1716836631182, experiment_id='0', last_update_time=1716836631182, lifecycle_stage='active', name='Default', tags={}>]

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

default artifacts URI: '/workspaces/mlops-camp/notebook/03-experiment-tracking/running-mlflow-exp/artifacts_local/1/9fc2196126f64c47ac049bd92156df12/artifacts'


In [5]:
mlflow.search_experiments()

[<Experiment: artifact_location='/workspaces/mlops-camp/notebook/03-experiment-tracking/running-mlflow-exp/artifacts_local/1', creation_time=1716837118065, experiment_id='1', last_update_time=1716837118065, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1716836631182, experiment_id='0', last_update_time=1716836631182, lifecycle_stage='active', name='Default', tags={}>]

In [6]:
from mlflow.tracking import MlflowClient

In [7]:
client = MlflowClient("http://127.0.0.1:5000")

In [8]:
client.search_registered_models()

[]

In [10]:
filter_string = "experiment_id='1'"  # Replace with your conditions

# List runs
run_infos = client.search_runs(experiment_ids=['1'])

# Access run information
for run_info in run_infos:
    run_id = run_info.info.run_id
    experiment_id = run_info.info.experiment_id
    # Access other run attributes as needed

    print(f"Run ID: {run_id}, Experiment ID: {experiment_id}")

Run ID: 9fc2196126f64c47ac049bd92156df12, Experiment ID: 1
Run ID: 4bc068d3b76d4757a8f75160cb186e10, Experiment ID: 1
Run ID: e300c19b4f46442a8003e483632affaa, Experiment ID: 1


In [13]:
run_info = client.search_runs(experiment_ids=['1'])[0]
print(run_info)
run_id = run_info.info.run_id
mlflow.register_model(
    model_uri=f"runs:/{run_id}/models",
    name='iris-classifier'
)

Successfully registered model 'iris-classifier'.
2024/06/04 18:14:32 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: iris-classifier, version 1


<Run: data=<RunData: metrics={'accuracy': 0.96}, params={'C': '0.1', 'random_state': '42'}, tags={'mlflow.log-model.history': '[{"run_id": "9fc2196126f64c47ac049bd92156df12", '
                             '"artifact_path": "models", "utc_time_created": '
                             '"2024-06-04 18:11:35.424151", "flavors": '
                             '{"python_function": {"model_path": "model.pkl", '
                             '"predict_fn": "predict", "loader_module": '
                             '"mlflow.sklearn", "python_version": "3.10.13", '
                             '"env": {"conda": "conda.yaml", "virtualenv": '
                             '"python_env.yaml"}}, "sklearn": '
                             '{"pickled_model": "model.pkl", '
                             '"sklearn_version": "1.4.1.post1", '
                             '"serialization_format": "cloudpickle", "code": '
                             'null}}, "model_uuid": '
                             '"1153

Created version '1' of model 'iris-classifier'.


<ModelVersion: aliases=[], creation_timestamp=1717524871978, current_stage='None', description='', last_updated_timestamp=1717524871978, name='iris-classifier', run_id='9fc2196126f64c47ac049bd92156df12', run_link='', source='/workspaces/mlops-camp/notebook/03-experiment-tracking/running-mlflow-exp/artifacts_local/1/9fc2196126f64c47ac049bd92156df12/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='1'>